In [1]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)

# current_dir = os.getcwd()
# os.environ['PROJECT_ROOT'] = os.path.join(current_dir, '..')

In [ ]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
config_paths = [
        '../config/dataset/gen1/event_frame/single/base.yaml',
        '../config/model/rvt_detector/rvt_frame.yaml',
        '../config/experiment/sequence/train.yaml',
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
dynamically_modify_train_config(config=merged_conf)

num_class 2
Set partition sizes: (8, 10)


In [7]:
from modules.fetch import fetch_data_module, fetch_model_module
import torch

data = fetch_data_module(merged_conf)
data.setup('fit')
model = fetch_model_module(merged_conf)
model.setup('fit')

ckpt_path = '../scripts/result/gen1/rvt-t/event_frame-dt50/20241116-134504/train/epoch=49-val_AP=0.42.ckpt'
ckpt = torch.load(ckpt_path, map_location=torch.device('cpu'))
model.load_state_dict(ckpt['state_dict'])

train dataset size: 49
valid dataset size: 58
rvt
RVT
PAFPN
neck input channels (64, 128, 256)
head strides (8, 16, 32)
YOLOX-Head


/var/folders/0f/5nhrfybd7r1d725z2nywhzg00000gn/T/ipykernel_60353/3336737903.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=to

<All keys matched successfully>

In [ ]:
import time
from utils.yolox_utils import to_yolox

rnn_model = model.model
rnn_model.eval()

state = None

# 全体の推論時間を計測する
start_time = time.time()

for i, batch in enumerate(data.val_dataloader()):
    # バッチごとの時間計測開始
    batch_start_time = time.time()
    
    events = batch['events'][:, 0].float()  # [batch, sequence_len, ch, h, w]
    labels = batch['labels']  # [batch, sequence_len, num, bbox]
    targets = to_yolox(labels, mode='train')[:, 0]

    # 推論
    outputs, state = rnn_model(events, state)
    
    # バッチごとの時間計測終了
    batch_end_time = time.time()
    batch_time = batch_end_time - batch_start_time
    # print(f"Batch {i} inference time: {batch_time:.4f} seconds")
    

# 全体の推論時間を計測終了
end_time = time.time()
total_time = end_time - start_time
print(f"Total inference time for {i+1} batches: {total_time:.4f} seconds")
